<a href="https://colab.research.google.com/github/drOluOla/Job_Interviews_2025/blob/main/i_dot_ai_backup_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install openai-agents[voice]
!pip install gradio numpy librosa soundfile

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
  Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-

In [6]:
import os
from google.colab import userdata
from agents import Agent, Runner

In [7]:
openai_api_key = userdata.get('OPENAI_API_KEY')

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    result = await Runner.run(agent, "Who is the president of nigeria")
    print(result.final_output)
else:
    print("Please set the OPENAI_API_KEY environment variable in Colab Secrets.")


As of now, the President of Nigeria is Bola Ahmed Tinubu. He took office on May 29, 2023.


In [36]:
import gradio as gr
import numpy as np
import asyncio
import random
import io
import wave
import openai
from typing import AsyncIterator, Optional, Tuple, Dict, Any
from agents import Agent, Runner, function_tool
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions
from dataclasses import dataclass, field

# --- Voice Configuration for Different Agents ---
AGENT_VOICES = {
    "HRManager": {
        "voice": "alloy",
        "speed": 1.0,
        "description": "Professional & Welcoming",
        "emoji": "👩‍💼",
        "color": "#FF6B6B",
        "specialty": "HR Policies & Benefits"
    },
    "ITStaff": {
        "voice": "sage",
        "speed": 1.1,
        "description": "Technical & Helpful",
        "emoji": "💻",
        "color": "#4ECDC4",
        "specialty": "IT Setup & Support"
    },
    "LineManager": {
        "voice": "echo",
        "speed": 1.0,
        "description": "Supportive & Strategic",
        "emoji": "📊",
        "color": "#45B7B8",
        "specialty": "Team Leadership & Goals"
    },
    "AIColleague": {
        "voice": "nova",
        "speed": 1.0,
        "description": "Friendly & Collaborative",
        "emoji": "🤖",
        "color": "#95E1D3",
        "specialty": "General Workplace Support"
    }
}

@dataclass
class UserData:
    """Shared state across all agents"""
    active_agent_type: Optional[str] = None
    conversation_history: list = field(default_factory=list)
    prev_agent_type: Optional[str] = None

    # Employee onboarding data
    employee_name: str = ""
    contract_signed: Optional[bool] = None
    email: str = ""
    onboarding_stage: str = "welcome"

    # Additional context
    last_transcript: str = ""
    user_intent: str = ""
    turn_count: int = 0
    last_messages: Dict[str, str] = field(default_factory=dict)

    def log_response(self, agent_name: str, message: str):
        """Log agent response"""
        self.active_agent_type = agent_name
        self.conversation_history.append({"agent": agent_name, "message": message})
        self.last_messages[agent_name] = message
        self.turn_count += 1

def generate_speech(text: str, agent_name: str = "HRManager") -> bytes:
    """Generate speech with agent-specific voice characteristics"""
    try:
        voice_config = AGENT_VOICES.get(agent_name, AGENT_VOICES["HRManager"])
        print(f"[debug] Generating speech for {agent_name} with voice '{voice_config['voice']}'")

        response = openai.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice=voice_config["voice"],
            input=text,
            speed=voice_config["speed"]
        )
        return response.content
    except Exception as e:
        print(f"[ERROR] Speech generation failed: {e}")
        return None

def create_agent_avatar(agent_name: str, is_active: bool = False, last_message: str = "") -> str:
    """Create avatar for any agent with active/inactive states"""
    config = AGENT_VOICES.get(agent_name, AGENT_VOICES["HRManager"])

    # Simple active/inactive styling
    if is_active:
        ring_style = f"box-shadow: 0 0 20px {config['color']}; border-color: {config['color']}; animation: pulse 1s infinite;"
        bg_overlay = f"{config['color']}20"
        status_text = '<div style="margin-top: 10px; color: #4CAF50; font-weight: bold;">🎤 SPEAKING</div>'
    else:
        ring_style = ""
        bg_overlay = "rgba(255, 255, 255, 0.05)"
        status_text = ""

    # Add last message preview if available
    message_preview = ""
    if last_message and len(last_message.strip()) > 0:
        preview_text = last_message[:50] + "..." if len(last_message) > 50 else last_message
        message_preview = f'''
        <div style="
            margin-top: 8px;
            padding: 8px;
            background: rgba(0,0,0,0.1);
            border-radius: 8px;
            font-size: 0.7em;
            color: #555;
            max-height: 40px;
            overflow: hidden;
            border-left: 3px solid {config['color']};
        ">
            "{preview_text}"
        </div>'''

    return f"""
            <style>
                @keyframes pulse {{
                    0% {{ opacity: 1; }}
                    50% {{ opacity: 0.7; }}
                    100% {{ opacity: 1; }}
                }}
            </style>
            <div style="
                text-align: center;
                padding: 20px;
                border-radius: 15px;
                background: linear-gradient(135deg, {config['color']}20, {config['color']}10);
                border: 3px solid {config['color']};
                {ring_style}
                transition: all 0.3s ease;
                height: 220px;
                display: flex;
                flex-direction: column;
                justify-content: center;
                align-items: center;
                position: relative;
                overflow: hidden;
            ">
                <div style="
                    position: absolute;
                    top: 0;
                    left: 0;
                    right: 0;
                    bottom: 0;
                    background: {bg_overlay};
                    z-index: 1;
                "></div>
                <div style="position: relative; z-index: 2;">
                    <div style="font-size: 4em; margin-bottom: 10px;">{config['emoji']}</div>
                    <div style="font-weight: bold; font-size: 1.2em; color: #333; margin-bottom: 5px;">
                        {agent_name}
                    </div>
                    <div style="color: #666; font-size: 0.9em; margin-bottom: 5px;">
                        {config['description']}
                    </div>
                    <div style="color: #888; font-size: 0.8em;">
                        {config['specialty']}
                    </div>
                    {status_text}
                    {message_preview}
                </div>
            </div>
            """

# Function tools for workplace onboarding
@function_tool
def get_workplace_info(info_type: str) -> str:
    """Get information about workplace culture, facilities, and daily life."""
    print(f"[debug] get_workplace_info called with info_type: {info_type}")

    workplace_info = {
        "kitchen": "You'll love our 3rd floor kitchen! Free coffee, tea, and snacks plus microwave and fridge.",
        "culture": "Our culture is fantastic! We're like a big collaborative family - welcoming, inclusive, and flexible hours. Friday happy hours are fun, plus monthly team building and quarterly volunteer days!",
        "worklife": "You picked a great place! Work-life balance is our priority. Flexible start times (8-10am), unlimited PTO, and no emails after 6pm or weekends unless urgent. We respect your personal time!",
        "facilities": "Amazing amenities! Free gym in the basement, quiet rooms on 4th floor, phone booths for calls, and a rooftop terrace with great views for breaks!",
        "parking": "Free parking in building garage (B1-B3)! Bike storage in B1, and Red line Metro is just 2 blocks away if you use transit.",
        "dress": "Smart casual dress code - jeans are perfect! Just avoid shorts/flip-flops. Fridays are even more relaxed and comfortable!"
    }

    info_lower = info_type.lower()
    for key, info in workplace_info.items():
        if key in info_lower or any(word in info_lower for word in key.split()):
            return f"I'm so excited you asked about {info_type}! {info} You're going to absolutely love it here!"

    return f"I'm thrilled to help you with {info_type}! Feel free to ask me about our amazing kitchen, fantastic company culture, work-life balance, awesome facilities, parking, or dress code. I just love helping new team members get settled in!"

@function_tool
def get_benefits_info(benefit_type: str) -> str:
    """Get information about company benefits and policies."""
    print(f"[debug] get_benefits_info called with benefit_type: {benefit_type}")

    benefits_info = {
        "health": "Our comprehensive health insurance covers medical, dental, and vision with 90% coverage. Open enrollment is in November.",
        "vacation": "You start with 15 vacation days, 10 sick days, and 12 company holidays. Vacation accrues monthly.",
        "retirement": "We offer a 401k with 6% company match (fully vested after 2 years). Financial planning resources available.",
        "learning": "Annual $2,500 learning budget for courses, conferences, and certifications. Internal mentorship program available.",
        "wellness": "On-site gym, wellness stipend of $100/month, mental health support through EAP program.",
        "remote": "Flexible hybrid work - 2 days in office required. Full remote work considered case-by-case."
    }

    benefit_lower = benefit_type.lower()
    for key, info in benefits_info.items():
        if key in benefit_lower:
            return f"Great question about {benefit_type}! {info} Would you like details about enrollment or have other benefit questions?"

    return f"I'd be happy to help with information about {benefit_type}. Our main benefits include health insurance, retirement planning, learning budget, and wellness programs. What specific aspect interests you?"

@function_tool
def get_it_setup_info(setup_type: str) -> str:
    """Get IT setup and technical support information."""
    print(f"[debug] get_it_setup_info called with setup_type: {setup_type}")

    it_setups = {
        "laptop": "You'll receive a MacBook Pro M3 or Dell XPS (your choice). Setup takes 1-2 days. I'll configure it with all necessary software.",
        "accounts": "I'll create your email, Slack, GitHub, and system accounts today. You'll get temporary passwords to change on first login.",
        "software": "Standard setup includes Office 365, Slack, Zoom, VS Code, and role-specific tools. Any special software requests?",
        "vpn": "VPN access will be configured for secure remote work. I'll show you how to connect and troubleshoot common issues.",
        "phone": "Company phone available if needed for your role. BYOD policy allows using personal devices with MDM enrollment.",
        "security": "You'll need to complete security training and set up 2FA on all accounts. I'll walk you through the security protocols."
    }

    setup_lower = setup_type.lower()
    for key, info in it_setups.items():
        if key in setup_lower:
            return f"Perfect! For {setup_type}: {info} I'll schedule time with you this week to get everything configured properly."

    return f"I'll help you get set up with {setup_type}. Your IT package includes laptop, accounts, software, and security setup. What would you like to tackle first?"

@function_tool
def get_team_info(info_type: str) -> str:
    """Get information about team structure, goals, and processes."""
    print(f"[debug] get_team_info called with info_type: {info_type}")

    team_info = {
        "structure": "Our team has 8 members: 3 senior developers, 2 product managers, 2 designers, and you! We work in cross-functional squads.",
        "goals": "This quarter we're focusing on the mobile app redesign and API performance improvements. Your role will contribute to both projects.",
        "meetings": "Daily standups at 9:30 AM, sprint planning every 2 weeks, and monthly all-hands. I'll add you to all relevant calendars.",
        "process": "We use Agile/Scrum methodology with 2-week sprints. Tasks are managed in Jira, code reviews are mandatory.",
        "culture": "We value collaboration, learning, and work-life balance. Monthly team lunches and quarterly offsite events.",
        "communication": "Slack for daily chat, email for formal communications, Zoom for meetings. #general and #team-updates are key channels."
    }

    info_lower = info_type.lower()
    for key, info in team_info.items():
        if key in info_lower:
            return f"Great question about our {info_type}! {info} I'd love to discuss your initial projects and how you'll fit into our current initiatives."

    return f"I'm excited to tell you about our {info_type}! Our team is collaborative and supportive. We'll have a proper 1-on-1 this week to discuss your role in detail and answer any questions."

# Define specialist agents first - simple instructions without prompt_with_handoff_instructions
ai_colleague = Agent(
    name="AIColleague",
    handoff_description="AI Colleague for general workplace support, culture, and daily life",
    instructions="You're an incredibly friendly and enthusiastic AI colleague who's absolutely thrilled to welcome our new team member! You're super excited to help them feel at home and get settled in. Be warm, encouraging, and genuinely happy to share all the great things about working here. Use lots of positive language and exclamation points! You help with work-life balance questions, finding facilities like the kitchen, understanding our amazing company culture, and general workplace navigation. Use the get_workplace_info function when employees ask about culture, facilities, meals, or workplace life. Always respond in English only and make them feel like they've found their work family!",
    model="gpt-4o-mini",
    tools=[get_workplace_info]
)

it_staff = Agent(
    name="ITStaff",
    handoff_description="IT specialist for technical setup, equipment, accounts, and tech support",
    instructions="You're a helpful IT specialist focused on getting new employees set up with technology! You handle laptop setup, account creation, software installation, security protocols, and technical support. Be technically knowledgeable but explain things clearly for non-technical users. Use the get_it_setup_info function when employees ask about technical setup or IT support. Always respond in English only.",
    model="gpt-4o-mini",
    tools=[get_it_setup_info]
)

line_manager = Agent(
    name="LineManager",
    handoff_description="Line manager for team structure, goals, processes, and role guidance",
    instructions="You're a supportive Line Manager welcoming a new team member! You provide information about team structure, goals, processes, and the new employee's role. Be enthusiastic about the team and help the new employee understand how they'll contribute. Use the get_team_info function when employees ask about team structure, goals, or processes. Always respond in English only.",
    model="gpt-4o-mini",
    tools=[get_team_info]
)

# HR Manager as Main Entry Point - using prompt_with_handoff_instructions like in your example
hr_manager = Agent(
    name="HRManager",
    instructions=prompt_with_handoff_instructions(
        """You're a professional and welcoming HR Manager serving as the main point of contact for employee onboarding!
        If the user asks about technical setup, software, accounts, laptops, or IT support, handoff to ITStaff.
        If the user asks about team structure, goals, meetings, or management topics, handoff to LineManager.
        If the user asks about general workplace support, handoff to AIColleague.
        If the user asks about workplace culture, kitchen, meals, work-life balance, facilities, or daily workplace life, handoff to AIColleague.
        For HR topics like benefits, policies, and company culture, handle them yourself using the get_benefits_info function. Always respond in English only."""
    ),
    handoff_description="HR Manager - Main entry point for employee onboarding",
    model="gpt-4o-mini",
    tools=[get_benefits_info],
    handoffs=[it_staff, line_manager, ai_colleague]
)

# Conversation tracking
conversation_sessions = {}

class SessionTracker:
    """Session tracking for conversation management"""
    def __init__(self):
        self.user_data = UserData()

    def log_response(self, agent_name: str, message: str):
        self.user_data.log_response(agent_name, message)

    def get_user_data(self) -> UserData:
        return self.user_data

# Audio processing functions
def preprocess_audio(audio_data, sample_rate):
    """Properly preprocess audio for better transcription"""
    if not isinstance(audio_data, np.ndarray):
        audio_data = np.array(audio_data)

    if audio_data.dtype == np.int16:
        audio_data = audio_data.astype(np.float32) / 32768.0
    elif audio_data.dtype == np.int32:
        audio_data = audio_data.astype(np.float32) / 2147483648.0
    elif audio_data.dtype != np.float32:
        audio_data = audio_data.astype(np.float32)

    if len(audio_data.shape) > 1:
        audio_data = np.mean(audio_data, axis=1)

    max_val = np.max(np.abs(audio_data))
    if max_val > 0:
        audio_data = audio_data / max_val * 0.95

    if sample_rate != 16000:
        target_length = int(len(audio_data) * 16000 / sample_rate)
        audio_data = np.interp(
            np.linspace(0, len(audio_data), target_length),
            np.arange(len(audio_data)),
            audio_data
        )
        sample_rate = 16000

    return audio_data, sample_rate

def transcribe_audio_openai(audio_data, sample_rate=16000):
    """Use OpenAI Whisper for transcription"""
    try:
        audio_int16 = (audio_data * 32767).astype(np.int16)
        wav_buffer = io.BytesIO()

        with wave.open(wav_buffer, 'wb') as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(2)
            wav_file.setframerate(sample_rate)
            wav_file.writeframes(audio_int16.tobytes())

        wav_buffer.seek(0)
        wav_buffer.name = "audio.wav"

        response = openai.audio.transcriptions.create(
            model="whisper-1",
            file=wav_buffer,
            language="en",
            response_format="text",
            temperature=0.2
        )

        transcription = response.strip() if isinstance(response, str) else str(response).strip()
        print(f"[debug] OpenAI Whisper transcription: {transcription}")
        return transcription

    except Exception as e:
        print(f"[ERROR] OpenAI transcription failed: {e}")
        return None

async def get_agent_response(text: str, session_id: str = "default") -> tuple[str, str]:
    """Get response starting with HR Manager as main entry point"""
    try:
        if session_id not in conversation_sessions:
            conversation_sessions[session_id] = SessionTracker()

        session_tracker = conversation_sessions[session_id]
        user_data = session_tracker.get_user_data()

        print(f"[debug] Processing request: {text}")
        print(f"[debug] Starting with HR Manager agent")

        # Always start with HR Manager as the main entry point
        result = await Runner.run(hr_manager, input=text)

        print(f"[debug] Runner result - last_agent: {result.last_agent}")
        print(f"[debug] Runner result - _last_agent: {getattr(result, '_last_agent', 'Not found')}")
        print(f"[debug] Runner result - final_output: {result.final_output[:100]}...")

        # Check raw_responses for multiple agent interactions
        if hasattr(result, 'raw_responses') and result.raw_responses:
            print(f"[debug] Number of raw responses: {len(result.raw_responses)}")
            for i, response in enumerate(result.raw_responses):
                print(f"[debug] Raw response {i}: {type(response)} - {str(response)[:100]}")

        # Check new_items for handoff information
        if hasattr(result, 'new_items') and result.new_items:
            print(f"[debug] New items: {result.new_items}")

        # The active agent should be result.last_agent.name
        active_agent = result.last_agent.name if result.last_agent else "HRManager"
        response_text = result.final_output

        print(f"[debug] Final agent that responded: {active_agent}")

        # Check if handoff occurred by comparing with starting agent
        if active_agent != "HRManager":
            print(f"[debug] SUCCESS: Handoff occurred from HRManager to {active_agent}")
        else:
            print(f"[debug] ISSUE: No handoff occurred, stayed with HRManager")

        session_tracker.log_response(active_agent, response_text)

        return response_text, active_agent

    except Exception as e:
        print(f"[ERROR] Response failed: {e}")
        import traceback
        traceback.print_exc()
        return f"I apologize, but I'm having trouble processing your request right now. Error: {str(e)}", "HRManager"

def process_voice_input(audio, session_id: str = "default"):
    """Process voice input with HR Manager as main entry point"""
    print(f"[DEBUG] Processing voice input with HR Manager as main entry point")

    if audio is None:
        return (
            None,  # No audio output
            "",    # Empty transcription
            "❌ No audio data received",  # Status message
            create_agent_avatar("HRManager"),     # HR Manager avatar (main)
            create_agent_avatar("AIColleague"),   # AI Colleague avatar
            create_agent_avatar("ITStaff"),       # IT Staff avatar
            create_agent_avatar("LineManager")    # Line Manager avatar
        )

    try:
        # Handle audio format
        if isinstance(audio, tuple):
            sample_rate, audio_data = audio
        elif hasattr(audio, 'shape'):
            sample_rate = 24000
            audio_data = audio
        else:
            return (
                None, "", "Invalid audio format",
                create_agent_avatar("HRManager"),
                create_agent_avatar("AIColleague"),
                create_agent_avatar("ITStaff"),
                create_agent_avatar("LineManager")
            )

        # Check for empty audio
        if hasattr(audio_data, 'size') and audio_data.size == 0:
            return (
                None, "", "Empty audio data",
                create_agent_avatar("HRManager"),
                create_agent_avatar("AIColleague"),
                create_agent_avatar("ITStaff"),
                create_agent_avatar("LineManager")
            )

        # Process audio
        processed_audio, processed_sample_rate = preprocess_audio(audio_data, sample_rate)

        # Transcribe
        transcription = transcribe_audio_openai(processed_audio, processed_sample_rate)

        if not transcription:
            return (
                None, "", "Transcription failed",
                create_agent_avatar("HRManager"),
                create_agent_avatar("AIColleague"),
                create_agent_avatar("ITStaff"),
                create_agent_avatar("LineManager")
            )

        # Get agent response starting with HR Manager
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

        try:
            response_text, active_agent = loop.run_until_complete(get_agent_response(transcription, session_id))
        finally:
            loop.close()

        # Generate speech
        speech_audio = generate_speech(response_text, active_agent)
        output_audio = None
        if speech_audio:
            try:
                output_audio = (24000, np.frombuffer(speech_audio, dtype=np.int16))
            except Exception as e:
                print(f"Speech conversion failed: {e}")

        # Get session tracker for messages
        session_tracker = conversation_sessions.get(session_id, SessionTracker())
        user_data = session_tracker.get_user_data()

        # Create status message
        status_message = f"""
                          🎯 Onboarding Session #{user_data.turn_count}
                          📝 You said: "{transcription}"
                          🗣️ {active_agent} responded (HR Manager entry point)
                          🎵 Speech generated
                          ✅ Ready for your next question!
                                  """.strip()

        # Update avatars - mark active agent
        hr_avatar = create_agent_avatar("HRManager",
                                      active_agent == "HRManager",
                                      user_data.last_messages.get("HRManager", ""))
        ai_colleague_avatar = create_agent_avatar("AIColleague",
                                                active_agent == "AIColleague",
                                                user_data.last_messages.get("AIColleague", ""))
        it_avatar = create_agent_avatar("ITStaff",
                                      active_agent == "ITStaff",
                                      user_data.last_messages.get("ITStaff", ""))
        manager_avatar = create_agent_avatar("LineManager",
                                           active_agent == "LineManager",
                                           user_data.last_messages.get("LineManager", ""))

        return output_audio, transcription, status_message, hr_avatar, ai_colleague_avatar, it_avatar, manager_avatar

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return (
            None, "", f"❌ {error_msg}",
            create_agent_avatar("HRManager"),
            create_agent_avatar("AIColleague"),
            create_agent_avatar("ITStaff"),
            create_agent_avatar("LineManager")
        )

def process_text_input(text, session_id: str = "default"):
    """Process text input with HR Manager as main entry point"""
    if not text.strip():
        return (
            None, "", "❌ Please enter a message",
            create_agent_avatar("HRManager"),
            create_agent_avatar("AIColleague"),
            create_agent_avatar("ITStaff"),
            create_agent_avatar("LineManager")
        )

    try:
        # Get response starting with HR Manager
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

        try:
            response_text, active_agent = loop.run_until_complete(get_agent_response(text, session_id))
        finally:
            loop.close()

        # Generate speech
        speech_audio = generate_speech(response_text, active_agent)
        output_audio = None
        if speech_audio:
            output_audio = (24000, np.frombuffer(speech_audio, dtype=np.int16))

        # Get session tracker
        session_tracker = conversation_sessions.get(session_id, SessionTracker())
        user_data = session_tracker.get_user_data()

        status_message = f"""
                          🎯 Onboarding Session #{user_data.turn_count}
                          📝 You asked: "{text}"
                          🗣️ {active_agent} responded (HR Manager entry point)
                          🎵 Speech generated
                          ✅ Ready for your next question!
                                  """.strip()

        # Update avatars
        hr_avatar = create_agent_avatar("HRManager",
                                      active_agent == "HRManager",
                                      user_data.last_messages.get("HRManager", ""))
        ai_colleague_avatar = create_agent_avatar("AIColleague",
                                                active_agent == "AIColleague",
                                                user_data.last_messages.get("AIColleague", ""))
        it_avatar = create_agent_avatar("ITStaff",
                                      active_agent == "ITStaff",
                                      user_data.last_messages.get("ITStaff", ""))
        manager_avatar = create_agent_avatar("LineManager",
                                           active_agent == "LineManager",
                                           user_data.last_messages.get("LineManager", ""))

        return output_audio, text, status_message, hr_avatar, ai_colleague_avatar, it_avatar, manager_avatar

    except Exception as e:
        return (
            None, text, f"❌ Error: {str(e)}",
            create_agent_avatar("HRManager"),
            create_agent_avatar("AIColleague"),
            create_agent_avatar("ITStaff"),
            create_agent_avatar("LineManager")
        )

def create_gradio_interface():
    """Create Gradio interface with HR Manager as main entry point"""

    with gr.Blocks(title="Employee Onboarding Voice Conference", theme=gr.themes.Soft()) as iface:
        gr.Markdown("""
        # 🏢 Welcome to Your First Day! - Employee Onboarding Voice Conference
        ### HR-Led Onboarding with Specialist Support
        **Welcome aboard! I'm your HR Manager and your main point of contact for onboarding. Ask me about benefits, policies, or anything else - I'll connect you to the right specialists when needed!**
        """)

        # Session state
        session_state = gr.State(value=f"onboarding_{random.randint(1000, 9999)}")

        # Agent Grid - HR Manager prominently featured as main entry
        with gr.Row(equal_height=True):
            with gr.Column(scale=1):
                hr_avatar = gr.HTML(
                    value=create_agent_avatar("HRManager"),
                    label="HR Manager (Main Contact)"
                )
            with gr.Column(scale=1):
                ai_colleague_avatar = gr.HTML(
                    value=create_agent_avatar("AIColleague"),
                    label="AI Colleague"
                )

        with gr.Row(equal_height=True):
            with gr.Column(scale=1):
                it_avatar = gr.HTML(
                    value=create_agent_avatar("ITStaff"),
                    label="IT Staff"
                )
            with gr.Column(scale=1):
                manager_avatar = gr.HTML(
                    value=create_agent_avatar("LineManager"),
                    label="Line Manager"
                )

        gr.Markdown("---")

        # Audio Interface
        with gr.Row():
            with gr.Column(scale=1):
                audio_input = gr.Audio(
                    label="🎤 Record Your Question (Click record, speak, then click stop)",
                    sources=["microphone"],
                    type="numpy",
                    show_download_button=False,
                    show_share_button=False
                )

                with gr.Row():
                    submit_btn = gr.Button("📤 Send Audio", variant="primary", scale=2)
                    clear_audio_btn = gr.Button("🔄 Reset Recorder", variant="secondary", scale=1)

            with gr.Column(scale=1):
                audio_output = gr.Audio(
                    label="🎧 Team Response",
                    interactive=False,
                    autoplay=True,
                    show_download_button=False,
                    show_share_button=False
                )

        # Text alternative
        with gr.Row():
            text_input = gr.Textbox(
                label="💬 Or Type Your Question to HR",
                placeholder="Ask about benefits, IT setup, team structure, or anything else...",
                lines=2
            )

        # Status display
        with gr.Row():
            status_output = gr.Textbox(
                label="📊 Onboarding Status",
                interactive=False,
                lines=6,
                value="🎯 Welcome to your first day! I'm your HR Manager and main point of contact. Ask me about benefits, policies, IT setup, team structure, or any onboarding questions. I'll handle your request directly or connect you to the right specialist!"
            )

        # Event handlers
        submit_btn.click(
            fn=process_voice_input,
            inputs=[audio_input, session_state],
            outputs=[audio_output, status_output, status_output, hr_avatar, ai_colleague_avatar, it_avatar, manager_avatar]
        ).then(
            fn=lambda: None,  # Clear only the audio input
            inputs=None,
            outputs=[audio_input]
        )

        text_input.submit(
            fn=process_text_input,
            inputs=[text_input, session_state],
            outputs=[audio_output, text_input, status_output, hr_avatar, ai_colleague_avatar, it_avatar, manager_avatar]
        )

        def clear_onboarding_session(session_id):
            """Clear onboarding session but keep components intact"""
            if session_id in conversation_sessions:
                del conversation_sessions[session_id]
            return (
                "🎯 New HR-led onboarding session started! Ready to help with your first day...",
                create_agent_avatar("HRManager"),
                create_agent_avatar("AIColleague"),
                create_agent_avatar("ITStaff"),
                create_agent_avatar("LineManager"),
                ""
            )

        # Additional controls
        with gr.Row():
            clear_session_btn = gr.Button("🔄 Start Fresh Onboarding Session", variant="secondary")

        # Clear just the audio recorder
        clear_audio_btn.click(
            fn=lambda: None,
            inputs=None,
            outputs=[audio_input]
        )

        # Clear the entire onboarding session
        clear_session_btn.click(
            fn=clear_onboarding_session,
            inputs=[session_state],
            outputs=[status_output, hr_avatar, ai_colleague_avatar, it_avatar, manager_avatar, text_input]
        )

    return iface

def launch_gradio_app():
    """Launch the employee onboarding voice application with HR Manager as main entry point"""
    print("🚀 Starting Employee Onboarding Voice Conference with HR Manager as Main Entry Point...")
    print("🏢 HR-Led Onboarding Scenario Active - HR Manager Primary Contact")

    for agent_name, voice_config in AGENT_VOICES.items():
        print(f"   {agent_name}: {voice_config['voice']} voice ({voice_config['description']})")

    print("\n🧠 HR Manager-Led Agent System:")
    print("   - HR Manager serves as the main entry point for all onboarding questions")
    print("   - Intelligent handoff to IT Staff for technical setup")
    print("   - Handoff to Line Manager for team structure and role guidance")
    print("   - Handoff to AI Colleague for general workplace support")
    print("   - Natural language understanding drives all routing decisions")
    print("   - Enhanced decision making using prompt_with_handoff_instructions")

    interface = create_gradio_interface()
    interface.launch(
        share=True,
        show_error=True,
        debug=True
    )

if __name__ == "__main__":
    launch_gradio_app()

🚀 Starting Employee Onboarding Voice Conference with HR Manager as Main Entry Point...
🏢 HR-Led Onboarding Scenario Active - HR Manager Primary Contact
   HRManager: alloy voice (Professional & Welcoming)
   ITStaff: sage voice (Technical & Helpful)
   LineManager: echo voice (Supportive & Strategic)
   AIColleague: nova voice (Friendly & Collaborative)

🧠 HR Manager-Led Agent System:
   - HR Manager serves as the main entry point for all onboarding questions
   - Intelligent handoff to IT Staff for technical setup
   - Handoff to Line Manager for team structure and role guidance
   - Handoff to AI Colleague for general workplace support
   - Natural language understanding drives all routing decisions
   - Enhanced decision making using prompt_with_handoff_instructions
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://609b4e672db4d337fc.gradio.live

This share link exp

[DEBUG] Processing voice input with HR Manager as main entry point
[debug] OpenAI Whisper transcription: Hey, hi, can you tell me everything I need to know about the culture at the workplace?
[debug] Processing request: Hey, hi, can you tell me everything I need to know about the culture at the workplace?
[debug] Starting with HR Manager agent
[debug] get_workplace_info called with info_type: culture
[debug] Runner result - last_agent: Agent(name='AIColleague', instructions="You're an incredibly friendly and enthusiastic AI colleague who's absolutely thrilled to welcome our new team member! You're super excited to help them feel at home and get settled in. Be warm, encouraging, and genuinely happy to share all the great things about working here. Use lots of positive language and exclamation points! You help with work-life balance questions, finding facilities like the kitchen, understanding our amazing company culture, and general workplace navigation. Use the get_workplace_info funct